# Impurity Model for nEXO

This Jupyter notebook was made so simply and easily plot the expected outgassing rate of a piece of material in a given system. 

## 1) Imports
We start off by importing a few necessary packages.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

Next we are goint to load the 3 pieces of code that contain all the information for this model. The first one is 'Library.py' which contains the parameters of various setups as well as materials to be used in that given setup. Before running this notebook you want to make sure that your combination of setup and material is entered correctly.

In [2]:
import Library as Lib 
import Outgassing as Out
from RunModel import * 

## 2) Define System
The class `System` will load the necessary information from `Library.py` by defining 4 key parameters: The name of the system (`Setup`), the material inside that setup that is outgassing (`Material`), the gas species that is dissolved inside the material (`Solute`) and the dimension of that piece of material (`Version`). 
In this example we want to plot the outgassing of oxygen from the piece of EXO-200 Teflon that we have at Yale. 

In [3]:
S1 = System(Setup='YLXPS', Material='Teflon', Solute='Oxygen', Version='EXO-Teflon')

You can check the values that go into getting the outgassing for this particular setup by executing

In [4]:
S1.Print() 

Name: YLXPS
 Material: Teflon
 Solute: Oxygen
 Version: EXO-Teflon
 Constraints: []
 ConstraintIndex: []
 Diffusion: 3.14e-07
 Solubility: 0.22
 ActivationEnergy: 0.47
 Abundance: 0.21
 MolarMass: 32
 XeMass: 2170
 Volume: 0.0053
 Area: 390.32180000000005
 Thickness: 0.15


Sets time regions for the above defined temperature values. The first element of Points is always 0, followed by the end of the first region and then the end of the second region. This example code will give you $\frac{100-0}{0.1}$ points from 0 to 100 and $\frac{200-100}{0.1}$ points from 100 to 200.

In [5]:
S1.Time = GetTimeStamps(Points=[0, 2, 20], Spacing=0.001, TimeScale='Days')

D:\nEXO_Impurities\RunModel.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(X)


Define the different temperatures for which to calculate outgassing. Note that `Temperature` should always be of type list even if only one value is used. Then add it to your setup. 

In [6]:
T1 = np.array([293.15]*len(S1.Time[0]))
T3 = np.array([164.0]*len(S1.Time[1]))
Temperature = [T1, T3]
S1.Temp = Temperature  

In [7]:
S1.Constraints = [1000000,10000000,1000000]

## 3) Calculate Outgassing
Calculate the diffusion constants for the above defined temperatures using the Arrhenius equation defined in `Outgassing.py`.

In [8]:
S1.DiffConstants = Out.GetDiffTemp(S1, Temperatures=S1.Temp, TimeScale='Days')    

C:\Users\Alex\miniconda3\envs\nexo\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

Get the initial number of impurities from model parameters. You are able to get the number of impurities in a variety of units. You can choose between total number of impurities (`'#'`), ratio between number of impurities and number of xenon atoms in the setup (`'ppm','ppb','ppt'`) or the total mass of impurities `'Mass'`. 

In [ ]:
S1.InitialImpurities = Out.GetInitialImpurities(S1, Units='#')

Define the labels for the plot. This should contain the name of the setup, the temperature and the thickness of the material.

In [ ]:
S1.Labels = ['%s, d = %s cm, T = %d K' % (S1.Version, S1.Thickness, 295), '%s, d = %s cm, T = %d K' % (S1.Version, S1.Thickness, 310)]

Calculate the number of impurities left in the sample as a function of time by using the solution to the diffusion equation defined in `Outgassing.py`

In [ ]:
S1.Impurities = Out.GetImpuritiesVsTime(Data=S1, TimeScale='Hours')

Calculate the outgassing rate as a function of time using Fick's 1st law, which is also defined in `Outgassing.py`

In [ ]:
S1.FlowRate = Out.GetFlowRateVsTime(Data=S1, Units='mBar Liter', TimeScale='Hours')

## 4) Plotting
Now one can easily plot the number of impurities left in the material and the outgassing rate expected from that material 

In [ ]:
PlotImpuritiesVsTime([S1], XRange=[0,20], YRange=[1E15, 1E18], XTicks=2, TimeScale='Days', Size=(10,6))
# PlotFlowRateVsTime([S1])

You can change the x- and y-ranges, respectively and modifiy the where to display the ticks. Lets change the x-range of our first plot to 0-10 hours and place x-ticks every 1 hour.

In [ ]:
PlotImpuritiesVsTime([S1], XRange=[0,10], YRange=[1E17, 1E18], XTicks=2, TimeScale='Days', Size=(10,6))

## 6) Run modelling for multiple setups, materials and gas solutes 
One can also easily define a few different combinations of materials, thicknesses and gases and plot them together. To do this just create an instance of the System class for each setup

In [ ]:
S1 = System(Setup='YLXPS', Material='Teflon', Solute='Oxygen', Version='EXO-Teflon')
S2 = System(Setup='YLXPS', Material='Teflon', Solute='Oxygen', Version='Stock-Teflon')
S3 = System(Setup='YLXPS', Material='Teflon', Solute='Oxygen', Version='Stock-Teflon Thick')

Then we can combine them into one list

In [ ]:
Systems = [S1,S2,S3]

Just like before we define the time ranges and the respective temperatures for each range.

In [ ]:
Temperature = [295, 340]
Time = GetTimeStamps(Points=[0,100,200], Spacing=0.1, TimeScale='Hours')

You can create the labels for each setup and temperature automatically by executing

In [ ]:
Labels = GetLabels(Systems, Temperature)

Now we have everything together to run the modelling to get the number of impurities and outgassing rates. This function combines all above steps and does them for each element in your list of `Systems`.

In [ ]:
DoModelling(Systems, Labels, Temperature, Time)

Finally, we can plot both number of impurities and outgassing rates for all setups together

In [ ]:
PlotImpuritiesVsTime(Systems)
PlotFlowRateVsTime(Systems, XRange=[0,100])

These plot can now be used to assess the effect of outgassing on the purity of a liquid xenon setup. 